In [36]:
from secret_key import OpenAIapi_key
import os
os.environ['OPENAI_API_KEY'] = OpenAIapi_key

In [49]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)
name = llm('Lead Actor name from Dilwale movie. then real-wife name of actor')

print(name)



Lead Actor: Shah Rukh Khan 
Real-wife: Gauri Khan


In [41]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['movie'],
    template = "Lead Actor name from {movie} ."
)

prompt_template_name.format(movie = 'URI')

'Lead Actor name from URI .'

In [47]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
print(chain.run('Super-30'))



Hrithik Roshan


In [93]:
llm = OpenAI(temperature = 0.9)

prompt_template_name = PromptTemplate(
    input_variables = ['movie'],
    template = "Lead Actor name from {movie} ."
)

first_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='actor_name')

prompt_template_name = PromptTemplate(
    input_variables = ['actor_name'],
    template = "Then wife name of {actor_name} ."
)

second_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key = 'wife_name' )

In [94]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [first_chain, second_chain])

response = chain.run('Dilwale')

print(response)



Gauri Khan


In [103]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [first_chain, second_chain],
    input_variables = ['movie'],
    output_variables = ['actor_name', 'wife_name']
    
)

chain({'movie': "Dilwale"})

{'movie': 'Dilwale',
 'actor_name': '\n\nShah Rukh Khan',
 'wife_name': '\n\nGauri Khan'}

## Agents

In [111]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run('when was ShahRukh Khan born? what is his age right now in 2023')



> Entering new AgentExecutor chain...
 I need to do a research on this
Action: Wikipedia
Action Input: ShahRukh Khan
Observation: Page: Shah Rukh Khan
Summary: Shah Rukh Khan (pronounced [ˈʃɑːɦɾʊx xɑːn]; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi films. Referred to in the media as the "Baadshah of Bollywood" and "King Khan", he has appeared in more than 90 films, and earned numerous accolades, including 14 Filmfare Awards. He has been awarded the Padma Shri by the Government of India, as well as the Ordre des Arts et des Lettres and Legion of Honour by the Government of France. Khan has a significant following in Asia and the Indian diaspora worldwide. In terms of audience size and income, several media outlets have described him as one of the most successful film stars in the world. Many of his films thematise Indian national identity and connections with diaspora communities, or gender, racial, social and religio

'Shah Rukh Khan was born on November 2nd 1965, and is now 58 years old in 2023.'

## Memory

In [118]:
from langchain.chains import ConversationChain

convo = ConversationChain(
    llm = OpenAI(temperature=0.6)
)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [119]:
convo.run('Who won the 2019 cricket world cup?')

' The 2019 cricket world cup was won by England. They defeated New Zealand in the final, with the game being decided on boundary count after a tied Super Over.'

In [120]:
convo.run('explain AI')

' AI stands for Artificial Intelligence. It is a field of computer science that focuses on creating machines that can think and act like humans. AI systems can be trained to analyze data, recognize patterns, make decisions, and even learn from their own experiences.'

In [121]:
convo.run('who was the captain of winning team?')

' The captain of the winning team in the 2019 cricket world cup was Eoin Morgan.'

In [122]:
print(convo.memory.buffer)

Human: Who won the 2019 cricket world cup?
AI:  The 2019 cricket world cup was won by England. They defeated New Zealand in the final, with the game being decided on boundary count after a tied Super Over.
Human: explain AI
AI:  AI stands for Artificial Intelligence. It is a field of computer science that focuses on creating machines that can think and act like humans. AI systems can be trained to analyze data, recognize patterns, make decisions, and even learn from their own experiences.
Human: who was the captain of winning team?
AI:  The captain of the winning team in the 2019 cricket world cup was Eoin Morgan.


In [123]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm = OpenAI(temperature=0.6),
    memory = memory
)
convo.run('Who won the 2015 cricket world cup?')

' The 2015 ICC Cricket World Cup was won by Australia. They beat New Zealand by 7 wickets in the final match.'

In [125]:
convo.run('why Agra famous?')

" Agra is best known for the iconic Taj Mahal, one of the seven wonders of the world and a UNESCO World Heritage Site. It is also home to several other monuments and historic sites, such as the Agra Fort, Fatehpur Sikri, and Sikandara Akbar's Tomb."

In [126]:
convo.run('who was the captain of winning team?')

" I don't know."

In [127]:
print(convo.memory.buffer)

Human: who was the captain of winning team?
AI:  I don't know.
